In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet

#set the random seeds
np.random.seed(42)
tf.random.set_seed(42)

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
#read the triplet pairs of images
train_triplets = pd.read_csv('train_triplets.txt', delimiter = ' ', names = ['A', 'B', 'C'], dtype=str) + '.jpg'
test_triplets = pd.read_csv('test_triplets.txt', delimiter = ' ', names = ['A', 'B', 'C'], dtype=str) + '.jpg'

#balance labels:
#add labels to the training dataframe
len = train_triplets.shape[0]
labels = np.ones(len)
train_triplets['labels'] = labels

#for the training data switch B and C random with 0.5 chance to balance labels
df_sample = train_triplets.sample(round(len/2))
indicies = df_sample.index
train_triplets.loc[indicies,'labels'] = 0

#dreiecks tausch:
df_B = train_triplets.loc[indicies, 'B']
train_triplets.loc[indicies,'B'] = train_triplets.loc[indicies,'C']
train_triplets.loc[indicies,'C'] = df_B

#read pre processed features
train_features = pd.read_csv('train_features_food.csv')
test_features = pd.read_csv('test_features_food.csv')

#standardization
scaler = StandardScaler()
train_features.iloc[:, 1:] = scaler.fit_transform(train_features.iloc[:, 1:])
test_features.iloc[:, 1:] = scaler.fit_transform(test_features.iloc[:, 1:])

In [3]:
def join_features(triplet,df):
    """
        for image names of dtype string this function extracts the features for the given images
        and returns the concatenated array
        
        Args:
            triplet: tupel of the image names in the triplet
            df: data frame from which we want to load the features
        returns:
            concatenated feature vector of the feature vectors of all three images
        
    """
    
    img_A = triplet[0]
    img_B = triplet[1]
    img_C = triplet[2]
    
    features_A = df[df['0'] == img_A].to_numpy()[0,1:]
    features_B = df[df['0'] == img_B].to_numpy()[0,1:]
    features_C = df[df['0'] == img_C].to_numpy()[0,1:]
    
    #concat features
    return np.concatenate([features_A,features_B,features_C])

In [4]:
num_triplets=train_triplets.shape[0]#number of triplets in the training set
num_triplets_test = test_triplets.shape[0]#number of triplets in the test set
triplets = train_triplets.to_numpy()[:,0:3]#training triplets
test_triplets = test_triplets.to_numpy()[:,0:3]#test triplets
lbs = train_triplets.to_numpy()[:,3]#labels 

In [5]:
#store the feature vector for all training triplets in a feature matrix 
train_features = np.array([join_features(triplets[i],train_features) for i in tqdm(range(num_triplets))])

100%|███████████████████████████████████| 59515/59515 [00:55<00:00, 1081.84it/s]


In [6]:
#store the feature vector for all test triplets in a feature matrix
test_features = np.array([join_features(test_triplets[i],test_features) for i in tqdm(range(num_triplets_test))])

100%|███████████████████████████████████| 59544/59544 [00:54<00:00, 1086.25it/s]


In [7]:
train_features = train_features.astype(float)
X_test_real = test_features.astype(float)

#train-test split
X_train, X_test, y_train, y_test = train_test_split(
    train_features, lbs, 
    test_size=0.1, random_state=42
)

y_train = y_train.astype(int)
X_train = X_train.astype(float)
y_test = y_test.astype(int)
X_test = X_test.astype(float)

In [38]:
#desing a model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(6144, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(3072, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1536, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(768, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(384, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(192, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [39]:
#compile a model
model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy')#,
        #tf.keras.metrics.Precision(name='precision'),
        #tf.keras.metrics.Recall(name='recall')
    ]
)

In [40]:
#initialize checkpoint-filepath for the best epoch 
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [41]:
#train the model
history = model.fit(X_train,
                    y_train,
                    epochs=10,
                    batch_size=128,
                    validation_data=(X_test, y_test),
                    callbacks=[model_checkpoint_callback])

Epoch 1/10


2022-05-11 01:31:39.891495: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


419/419 [==============================] - ETA: 0s - loss: 0.5560 - accuracy: 0.7228

2022-05-11 01:32:07.043241: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


419/419 [==============================] - 29s 64ms/step - loss: 0.5560 - accuracy: 0.7228 - val_loss: 0.5205 - val_accuracy: 0.7465
Epoch 2/10
419/419 [==============================] - 24s 57ms/step - loss: 0.4369 - accuracy: 0.7978 - val_loss: 0.5330 - val_accuracy: 0.7515
Epoch 3/10
419/419 [==============================] - 24s 58ms/step - loss: 0.3556 - accuracy: 0.8445 - val_loss: 0.5017 - val_accuracy: 0.7749
Epoch 4/10
419/419 [==============================] - 24s 58ms/step - loss: 0.2845 - accuracy: 0.8797 - val_loss: 0.5030 - val_accuracy: 0.8041
Epoch 5/10
419/419 [==============================] - 24s 57ms/step - loss: 0.2201 - accuracy: 0.9116 - val_loss: 0.5622 - val_accuracy: 0.7866
Epoch 6/10
419/419 [==============================] - 24s 58ms/step - loss: 0.1711 - accuracy: 0.9324 - val_loss: 0.6628 - val_accuracy: 0.7708
Epoch 7/10
419/419 [==============================] - 24s 58ms/step - loss: 0.1396 - accuracy: 0.9455 - val_loss: 0.6354 - val_accuracy: 0.8026
Epo

In [42]:
#reload the best model
model.load_weights(checkpoint_filepath)

#print accuracy on the validation set of the best model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

  4/186 [..............................] - ETA: 4s - loss: 0.4247 - accuracy: 0.8438 

2022-05-11 01:36:16.385668: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


186/186 [==============================] - 1s 6ms/step - loss: 0.5660 - accuracy: 0.8384
Accuracy: 83.84


In [43]:
#evaluate predicitons for the test set
predictions = (model.predict(X_test_real) > 0.5).astype(int)

2022-05-11 01:36:24.027758: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [44]:
#store the predictions
preds = predictions.reshape(-1).astype(int)
df_handin = pd.DataFrame({'predicitons': preds})
df_handin.to_csv('predictions_final.txt',index=False, header = False)